In [2]:
import pandas as pd
import spacy
from spacy import displacy
from spacy.tokens import Token
from spacy import Language
from spacy.matcher import Matcher
import en_core_web_lg


In [8]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("en_core_web_lg")

In [3]:
!pip install <model_s3_url> # tar.gz file e.g. from release notes like https://github.com/explosion/spacy-models/releases//tag/en_core_web_lg-2.3.1
!pip install en_core_web_lg

Het systeem kan het opgegeven bestand niet vinden.


In [5]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

In [25]:
data = pd.read_excel('../Assets/preprocessed_text.xlsx')

In [26]:
data[data['ids']=='test/14839']

,ids,categories,text
5,test/14839,['ship'],"Tug crews in New South Wales (NSW), Victoria ..."


In [27]:
text = data.iloc[5,2]

In [5]:

nlp = spacy.load("en_core_web_sm")
text = "I am Ujjwal"
doc = nlp(text)
info = []
for token in doc:
    info.append([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop])

tokens_df = pd.DataFrame(info,columns=["text","lemma","POS","tag","dep","shape","is_alpha","is_stop"])

In [40]:
displacy.render(doc, style='ent')


In [43]:
Token.set_extension('ref_n', default='', force = True)
Token.set_extension('ref_t', default='', force = True)

@Language.component("init_coref")
def init_coref(doc):
    for e in doc.ents:
        if e.label_ in ['ORG', 'GOV', 'PERSON']:
            e[0]._.ref_n, e[0]._.ref_t = e.text, e.label_
    return doc
init_coref(doc)

Tug crews in New South Wales (NSW),  Victoria and Western Australia yesterday lifted their ban on  foreign-flag ships carrying containers but NSW ports are still  being disrupted by a separate dispute, shipping sources said.  The ban, imposed a week ago over a pay claim, had prevented  the movement in or out of port of nearly 20 vessels, they said.  The pay dispute went before a hearing of the Arbitration  Commission today.  Meanwhile, disruption began today to cargo handling in the  ports of Sydney, Newcastle and Port Kembla, they said.  The industrial action at the NSW ports is part of the week  of action called by the NSW Trades and Labour Council to  protest changes to the states workers compensation laws.  The shipping sources said the various port unions appear to  be taking it in turn to work for a short time at the start of  each shift and then to walk off.  Cargo handling in the ports has been disrupted, with  container movements most affected, but has not stopped  altogether,

In [45]:

matcher = Matcher(nlp.vocab)

acq_synonyms = ['acquire', 'buy', 'purchase']
pattern = [{'_': {'ref_t': 'ORG'}}, # subject
           {'_': {'ref_t': {'NOT_IN': ['ORG']}}, 'OP': '*'},
           {'POS': 'VERB', 'LEMMA': {'IN': acq_synonyms}},
           {'_': {'ref_t': {'NOT_IN': ['ORG']}}, 'OP': '*'},
           {'_': {'ref_t': 'ORG'}}] # object
matcher.add('acquires', [pattern])